In [7]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
# cap = cv2.VideoCapture(0) #This line will only be used for Live detection
def handGesture(img):
  # ret,img = cap.read()
  img = cv2.resize(img, (int(img.shape[1]/4), int(img.shape[0]/4)))
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray,(5,5),0)
  ret,thresh1 = cv2.threshold(blur,70,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

  contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  drawing = np.zeros(img.shape,np.uint8)

  max_area=0
  
  for i in range(len(contours)):
    cnt=contours[i]
    area = cv2.contourArea(cnt)
    if (area>max_area):
      max_area=area
      ci=i
  cnt=contours[ci]
  hull = cv2.convexHull(cnt)
  moments = cv2.moments(cnt)
  if moments['m00']!=0:
    cx = int(moments['m10']/moments['m00']) # cx = M10/M00
    cy = int(moments['m01']/moments['m00']) # cy = M01/M00
            
  centr=(cx,cy)       
  cv2.circle(img,centr,5,[0,0,255],2)       
  cv2.drawContours(drawing,[cnt],0,(0,255,0),2) 
  cv2.drawContours(drawing,[hull],0,(0,0,255),2) 
        
  cnt = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
  hull = cv2.convexHull(cnt,returnPoints = False)
  
  if (1):
    defects = cv2.convexityDefects(cnt,hull)
    mind=0
    maxd=0
    for i in range(defects.shape[0]):
      s,e,f,d = defects[i,0]
      start = tuple(cnt[s][0])
      end = tuple(cnt[e][0])
      far = tuple(cnt[f][0])
      dist = cv2.pointPolygonTest(cnt,centr,True)
      cv2.line(img,start,end,[0,0,255],5)
      
      cv2.circle(img,far,10,[0,0,255],-1)
      font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(img, "Number of Fingers: "+str(i), ( 26, 106), font, 1, (0, 0, 255), 2)
    return (str(i))

In [9]:
input = cv2.imread("/content/6 fingers.jpeg")
output = handGesture(input)
'''cv2_imshow'''
print(output)

6


In [10]:
pip install pyngrok

In [11]:
!pip install flask_ngrok

In [12]:
'''Use your own authentication token'''
!ngrok authtoken 22EGVz9Y7DYxPz1Ku9gOLTLFn4i_GyaLYHDKnQH9PT5VuLHY

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [13]:
import numpy as np
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import cv2

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)   


@app.route("/", methods=['GET', 'POST'])
def main():
  return render_template("index.html")

@app.route("/submit", methods = ['GET', 'POST'])
def get_output():
  if request.method == 'POST':
    img = request.files['my_image']
    img_path = "static/" + img.filename  
    img.save(img_path)
    print(img_path)
    img=cv2.imread(img_path)
    p = handGesture(img)
    return render_template("index.html", prediction = p, img_path = img_path)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bee4-104-196-163-228.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Dec/2021 11:54:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 11:54:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 11:54:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Dec/2021 11:54:37] "POST /submit HTTP/1.1" 200 -


static/6 fingers.jpeg


127.0.0.1 - - [27/Dec/2021 11:54:37] "GET /static/6%20fingers.jpeg HTTP/1.1" 200 -
[2021-12-27 11:54:39,490] ERROR in app: Exception on /submit [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [27/Dec/2021 11:54:39] "GET /submit HTTP/1.1" 500 -
